In [ ]:
from datetime import date, timedelta, datetime
from requests import request
from urllib.parse import quote
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

In [ ]:
def text2date(q):
    if "segundo" in q:
        return (date.today().isoformat())
    elif "minuto" in q:
        return (date.today().isoformat())
    elif "hora" in q:
        return (date.today().isoformat())
    elif "hoje" in q:
        return (date.today().isoformat())
    elif "Ontem" in q:
        return (date.today()-timedelta(days=1)).isoformat()
    elif "dia" in q:
        return (date.today()-timedelta(days=int(q[:2]))).isoformat()
    elif "de" in q:
        q=q.replace("abr","apr").replace("mai","may").replace("ago","aug").replace("set","sep").replace("out","oct").replace("dez","dec")
        q=q+str(date.today().year)
        return datetime.strptime(q,"%d de %b.%Y").date().isoformat()
    else:
        return q

In [ ]:
def getNews(termo: str, limite=10, dias=30) -> list:
    requestHeader = { "user-agente":"Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    url = "https://news.google.com/search?q="+quote(termo)+"%20when%3A"+str(dias)+"d&hl=pt-BR&gl=BR&ceid=BR%3Apt-419"
    r = request('get',url=url, headers=requestHeader)
    dados =[]
    if(r.status_code==200):
        soup = BeautifulSoup(r.content, 'html.parser')
        artigos = soup.find_all('article')
        for count,artigo in enumerate(artigos):
            if(count>=limite):
                break
            try:
                noticia = artigo.h3.text
                origem = artigo.div.a.text
                quando = text2date(artigo.div.time.text)
                link="https://news.google.com"+artigo.a['href'][1:]
                try:
                    r = request('get',url=link, headers=requestHeader)
                    if(r.status_code==200):
                        link=r.url
                except:
                    pass
                dados.append([termo,noticia,origem,quando,link])
            except:
                pass
    if len(dados) == 0:
        dados.append([termo,'* NÃO ENCONTRADA OCORRÊNCIA'])
    return dados

In [ ]:
def fastgnews(termos=[]) -> pd.DataFrame:
    with tqdm(total=len(termos),desc="Pesquisas") as pbar:  
        with ThreadPoolExecutor(max_workers=8) as executor:
            futures = [executor.submit(getNews, arg) for arg in termos]
            news=[]
            for future in as_completed(futures):
                news.extend(future.result())
                pbar.update(1)  
    df = pd.DataFrame(news,columns=["termo","noticia","origem","quando","link"])
    return df

Aqui começa a utilização


In [ ]:
sufix = ["acusado", "investigado","denunciado"]
termos = ["lula","bolsonaro"]
pesquisa = [ t+" , "+s for t in termos for s in sufix ]

In [ ]:
%%time
df=fastgnews(pesquisa)
df.sample(5)

In [ ]:
len(df)
#df.to_csv("saida_pesquisa.csv",sep=";")